In [18]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from SmartTruck import SmartTruck
from kalmanfilter2 import KalmanFilter

In [2]:
output = np.empty((4, 1))
output

array([[9.23057466e-312],
       [5.76887126e-313],
       [9.23019401e-312],
       [6.73134807e+175]])

In [3]:
ds = 3
t = 100 # number of time steps to run
dt = 0.1 # time step
ep = 0.1 # process noise variation
nu = 0.4 # measurement noise variation (both position and velocity)
initial = np.ones(ds*2)
initial.shape = (ds*2,1)

gen = SmartTruck(initial, t, dt, ep, nu)
process = gen.process()
measure = gen.measure(process)

In [12]:
output = np.empty((ds*2, 1))
z = np.zeros((ds,ds))
Q = np.block([[z,z],[z,np.eye(ds)*ep]])
R = np.eye(ds) * nu
filter = KalmanFilter(initial, gen.A, Q, R, gen.H)
for i in range(t+1):
    measure_t = measure[:,i]
    measure_t.shape = (ds,1)
    filter.predict(measure_t)
    kalman_output = filter.get_current_guess()
    output = np.append(output, kalman_output, axis=1)

In [22]:
%matplotlib
ax = plt.axes(projection='3d')

sz = 5
a = 0.8
ax.scatter3D(process[0], process[1], process[2], s=sz, alpha=a)
ax.scatter3D(measure[0], measure[1], measure[2], s=sz, alpha=a)
ax.scatter3D(output[0], output[1], output[2], s=sz, alpha=a, color='black')
plt.title("Position")
plt.xlabel("x")
plt.ylabel("y")
plt.legend(["Process","Measure","Filter"])

Using matplotlib backend: Qt5Agg
